 # Opis notatnika
 Zmierzamy do końca analizy danych, które zostały nam udostępnione. Ten krok dodaje jeszcze więcej informacji do naszego wyjściowego zbioru. Tym razem sprawdzimy między innymi to, czy opóźnienia lotów zależne są od trasy czy warunków pogodowych.

 Zanim jednak do tego przejdziemy, należy, podobnie jak w poprzednich krokach, skonfigurować odpowiednio notatnik.
 
 W tej części warsztatu ponownie wcielasz się w rolę Analiyka Danych, którego zadaniem jest wykonanie analizy eksplotacyjnej zbioru danych - jedno z wymagań dostarczonych przez klienta.

 Tutaj zaimportuj wymagane biblioteki

In [24]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, text as sql_query
import plotly.express as px
import matplotlib.pyplot as plt
import os
import datetime as dt

 ## Połączenie z bazą danych
 Tutaj uzupełnij konfigurację połączenia

In [25]:
username = 'postgres'
password = '123'
url = 'localhost'
host = 'localhost'
database = 'airlines'
port = None

 Tutaj stwórz zmienną engine, która zostanie użyta do połączenia z bazą danych

In [26]:
url = 'localhost'
engine = create_engine(f"postgresql://{username}:{password}@{url}/{database}")
connection = engine.connect()

 Tutaj uzupełnij implementację metody `read_sql_table`

In [27]:
def read_sql_table(table_name):
    pass
    df = pd.read_sql_table(table_name,  con= connection)
    return df

 Tutaj zaczytaj zapisaną wcześniej ramkę danych `flight_df` do zmniennej o takiej samej nazwie

In [28]:
os.chdir('../data/processed')
flight_04_03_df = pd.read_csv('flight_df_04_02.csv')
os.chdir('../../notebooks')
flight_04_03_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1026767 entries, 0 to 1026766
Data columns (total 32 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   month                 1026767 non-null  int64  
 1   day_of_month          1026767 non-null  int64  
 2   day_of_week           1026767 non-null  int64  
 3   op_unique_carrier     1026767 non-null  object 
 4   tail_num              1026767 non-null  object 
 5   op_carrier_fl_num     1026767 non-null  int64  
 6   origin_airport_id     1026767 non-null  int64  
 7   dest_airport_id       1026767 non-null  int64  
 8   crs_dep_time          1026767 non-null  int64  
 9   dep_time              1026767 non-null  float64
 10  dep_delay             1026767 non-null  float64
 11  dep_time_blk          1026767 non-null  object 
 12  crs_arr_time          1026767 non-null  int64  
 13  arr_time              1026574 non-null  float64
 14  arr_delay_new         1024912 non-

 # Wzbogacenie o `airport_list`
 Wczytaj do obszaru roboczego tabelę `airport_list` używając procedury `read_sql_table`. Wykonaj poniższe ćwiczenia:  
 1. Sprawdź, czy klucz `origin_airport_id` jest unikalny, tj. nie ma dwóch takich samych wartości w kolumnie `origin_airport_id`.  
 1. Jeżeli duplikaty występują, usuń je w najdogodniejszy dla Ciebie sposób.  
 1. Jeśli duplikaty nie występują, złącz ramki `airport_list_df` wraz z aktualną `flight_df`, używając kolumny `origin_airport_id` oraz złączenia typu `LEFT JOIN`. Z ramki `airport_list_df` interesuje nas dodanie kolumny `origin_city_name`.  
 1. Dodatkowo dokonaj jeszcze raz złączenia ramki `flight_df` z `airport_list_df`, tym razem jednak złącz kolumnę `destination_airport_id` wraz z `origin_airport_id`. Podobnie jak wcześniej, interesuje nas kolumna `origin_city_name`, jedank ona powinna zostać wyświetlona jako `destination_city_name`

 Tutaj wczytaj ramkę `airport_list_df`

In [6]:
airport_list_df = pd.read_sql('airport_list', connection)
airport_list_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   origin_airport_id     97 non-null     int64 
 1   display_airport_name  97 non-null     object
 2   origin_city_name      97 non-null     object
 3   name                  97 non-null     object
dtypes: int64(1), object(3)
memory usage: 3.2+ KB


In [7]:
len(airport_list_df['origin_airport_id'].unique())

97

***'origin_airport_id' jest kluczem unikalnym bo na 97 wierszy w tabeli jest 97 kluczy unikalnych w airport_list_df***

 Tutaj sprawdż, czy występują duplikaty dla kolumny `origin_airport_id`

In [8]:
airport_list_df.duplicated().value_counts()

False    97
dtype: int64

 Tutaj usuń duplikaty – jeśli występują

In [9]:
# nie występują

 Tutaj dokonaj złączenia ramki `flight_df` oraz `airport_list_df` używając `origin_airport_id` 
 Jeśli duplikaty nie występują, złącz ramki airport_list_df wraz z aktualną flight_df, używając kolumny origin_airport_id oraz złączenia typu LEFT JOIN. Z ramki airport_list_df interesuje nas dodanie kolumny origin_city_name.

In [10]:
flight_df_merged = pd.merge(flight_04_03_df, airport_list_df[['origin_airport_id','origin_city_name']],   how = 'left' ,\
                            on = 'origin_airport_id')
flight_df_merged.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1026767 entries, 0 to 1026766
Data columns (total 33 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   month                 1026767 non-null  int64  
 1   day_of_month          1026767 non-null  int64  
 2   day_of_week           1026767 non-null  int64  
 3   op_unique_carrier     1026767 non-null  object 
 4   tail_num              1026767 non-null  object 
 5   op_carrier_fl_num     1026767 non-null  int64  
 6   origin_airport_id     1026767 non-null  int64  
 7   dest_airport_id       1026767 non-null  int64  
 8   crs_dep_time          1026767 non-null  int64  
 9   dep_time              1026767 non-null  float64
 10  dep_delay             1026767 non-null  float64
 11  dep_time_blk          1026767 non-null  object 
 12  crs_arr_time          1026767 non-null  int64  
 13  arr_time              1026574 non-null  float64
 14  arr_delay_new         1024912 non-

 Tutaj dokonaj złączenia ramki `flight_df` oraz `airport_list_df` używając `destination_airport_id`

# sprawdzic mnazwy kolumn

In [11]:
# airport_list_df.head()
flight_df_merged.head()

,month,day_of_month,day_of_week,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,...,weather_delay,nas_delay,security_delay,late_aircraft_delay,is_delayed,is_weekend,distance_agg,manufacture_year,manufacture_year_agg,origin_city_name
0,1,20,7,WN,N204WN,682,10397,11292,605,602.0,...,NaN,NaN,NaN,NaN,False,True,"(1100, 1200]",2005.0,"(2002, 2005]","Atlanta, GA"
1,1,20,7,WN,N8682B,2622,10397,11292,2120,2114.0,...,NaN,NaN,NaN,NaN,False,True,"(1100, 1200]",2016.0,"(2014, 2017]","Atlanta, GA"
2,1,20,7,WN,N717SA,2939,10397,11292,1800,1807.0,...,0.0,10.0,0.0,3.0,True,True,"(1100, 1200]",1998.0,"(1996, 1999]","Atlanta, GA"
3,1,20,7,WN,N709SW,3848,10397,11292,1355,1354.0,...,NaN,NaN,NaN,NaN,False,True,"(1100, 1200]",1998.0,"(1996, 1999]","Atlanta, GA"
4,1,20,7,WN,N7864B,1352,10397,11697,1125,1125.0,...,NaN,NaN,NaN,NaN,False,True,"(500, 600]",2001.0,"(1999, 2002]","Atlanta, GA"


In [12]:
flight_df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1026767 entries, 0 to 1026766
Data columns (total 33 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   month                 1026767 non-null  int64  
 1   day_of_month          1026767 non-null  int64  
 2   day_of_week           1026767 non-null  int64  
 3   op_unique_carrier     1026767 non-null  object 
 4   tail_num              1026767 non-null  object 
 5   op_carrier_fl_num     1026767 non-null  int64  
 6   origin_airport_id     1026767 non-null  int64  
 7   dest_airport_id       1026767 non-null  int64  
 8   crs_dep_time          1026767 non-null  int64  
 9   dep_time              1026767 non-null  float64
 10  dep_delay             1026767 non-null  float64
 11  dep_time_blk          1026767 non-null  object 
 12  crs_arr_time          1026767 non-null  int64  
 13  arr_time              1026574 non-null  float64
 14  arr_delay_new         1024912 non-

In [13]:
flight_df_merged_2 = pd.merge(flight_df_merged, airport_list_df[['origin_airport_id','origin_city_name']],\
                              how = 'left' , left_on = 'dest_airport_id',\
                              right_on = 'origin_airport_id')

flight_df_merged_2.rename(columns = {'origin_airport_id_x':'origin_airport_id',\
                                   'origin_city_name_x':'origin_city_name',\
                                   'origin_city_name_y':'dest_city_name'}, inplace = True)
flight_df_merged_2.drop(columns = 'origin_airport_id_y', inplace =True)

In [14]:
flight_df_merged_2.head()

,month,day_of_month,day_of_week,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,...,nas_delay,security_delay,late_aircraft_delay,is_delayed,is_weekend,distance_agg,manufacture_year,manufacture_year_agg,origin_city_name,dest_city_name
0,1,20,7,WN,N204WN,682,10397,11292,605,602.0,...,NaN,NaN,NaN,False,True,"(1100, 1200]",2005.0,"(2002, 2005]","Atlanta, GA","Denver, CO"
1,1,20,7,WN,N8682B,2622,10397,11292,2120,2114.0,...,NaN,NaN,NaN,False,True,"(1100, 1200]",2016.0,"(2014, 2017]","Atlanta, GA","Denver, CO"
2,1,20,7,WN,N717SA,2939,10397,11292,1800,1807.0,...,10.0,0.0,3.0,True,True,"(1100, 1200]",1998.0,"(1996, 1999]","Atlanta, GA","Denver, CO"
3,1,20,7,WN,N709SW,3848,10397,11292,1355,1354.0,...,NaN,NaN,NaN,False,True,"(1100, 1200]",1998.0,"(1996, 1999]","Atlanta, GA","Denver, CO"
4,1,20,7,WN,N7864B,1352,10397,11697,1125,1125.0,...,NaN,NaN,NaN,False,True,"(500, 600]",2001.0,"(1999, 2002]","Atlanta, GA","Fort Lauderdale, FL"


***pozostawiam to jako ćwiczenie bo fajna metoda na mapowanie wartości jednej kolumny na drugą DODAJE KOLUMNE DEST_CITY_NAME NA PODSTAWIE DEST_AIRPORT_ID przez mapowanie***

In [15]:
# # flight_df
# dest_city_name_dict = flight_df.set_index('origin_airport_id_x')['origin_city_name'].to_dict()
# dest_city_name_dict
# flight_df['dest_city_name'] = flight_df['dest_airport_id'].map(dest_city_name_dict)
# flight_df = flight_df.rename(columns = {'Unnamed: 0':'count'})
# flight_df

### Sprawdzenie
Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [16]:
assert 'origin_city_name' in flight_df_merged_2.columns, 'Brak kolumny `origin_city_name` w ramce flight_df'
assert 'dest_city_name' in flight_df_merged_2.columns, 'Brak kolumny `dest_city_name` w ramce flight_df'

flight_df_expected_rows_amount = 1057391
assert flight_df_merged.shape[0] == flight_df_expected_rows_amount, 'Ups, zmieniła się liczba wierszy...'

AssertionError: Ups, zmieniła się liczba wierszy...

 ## Analiza według lotnisk oraz tras
 Wykonaj poniższe polecenia:  
 1. Wyznacz lotniska, z których **odlatywało** najwięcej samolotów. Wynik zapisz do ramki `top_airports_origin_df`.
 1. Wyznacz lotnika, na których najwięcej lotów **się kończyło**. Wynik zapisz do ramki `top_airports_destination_df`.  
 1. Wyznacz najczęściej uczęszczaną trasę, wynik zapisz do ramki `top_route_df`.  
 1. Przy założeniu, że reprezentatywna liczba lotów na trasie wynosi ponad 500, wyznacz dodatkowo top 10:  
     - tras z **najmniejszym odsetkiem opóźnień**, wynik zapisz do ramki `least_route_delays_df`.  
     - tras z **największym odsetkiem opóźnień**, wynik zapisz do ramki `top_route_delays_df`.

 Tutaj wyznacz ramkę `top_airports_origin_df`

In [17]:
# Wyznacz lotniska, z których odlatywało najwięcej samolotów. Wynik zapisz do ramki top_airports_origin_df.
top_airports_origin_df = flight_df_merged_2.groupby('origin_city_name')['origin_airport_id'].\
                                                agg('count').reset_index().sort_values('origin_airport_id',ascending = False)

top_airports_origin_df.head(10)

,origin_city_name,origin_airport_id
1,"Atlanta, GA",123162
6,"Chicago, IL",105437
19,"Los Angeles, CA",75052
10,"Denver, CO",64525
23,"New York, NY",63049
3,"Boston, MA",55989
34,"Seattle, WA",50465
32,"San Francisco, CA",50124
9,"Dallas/Fort Worth, TX",39511
17,"Las Vegas, NV",38456


 Tutaj wyznacz ramkę `top_airports_destination_df`

In [18]:
top_airports_destination_df = flight_df_merged_2.groupby('dest_city_name')['dest_airport_id'].\
                                                agg('count').reset_index().sort_values('dest_airport_id',ascending = False)
top_airports_destination_df.head(10)

,dest_city_name,dest_airport_id
1,"Atlanta, GA",122945
6,"Chicago, IL",100333
19,"Los Angeles, CA",75040
10,"Denver, CO",64602
24,"New York, NY",62972
3,"Boston, MA",56057
36,"Seattle, WA",50230
33,"San Francisco, CA",49999
9,"Dallas/Fort Worth, TX",39488
17,"Las Vegas, NV",38494


Wyznacz najczęściej uczęszczaną trasę, wynik zapisz do ramki top_route_df.
Przy założeniu, że reprezentatywna liczba lotów na trasie wynosi ponad 500, wyznacz dodatkowo top 10:
tras z najmniejszym odsetkiem opóźnień, wynik zapisz do ramki least_route_delays_df.
tras z największym odsetkiem opóźnień, wynik zapisz do ramki top_route_delays_df.

# ***NIE MAM POJĘCIA JAK TO ZROBIĆ...PONIŻEJ DWIA PODEJŚCIA ALE NIE MOGE DOJŚC DO TEGO JAK POKAZAĆ NUMER LOTU, MIASTO STARTOWE, MIASTO DOCELOWE,  SUME PRZELOTÓW,  NIE WIEM DLACZEGO MAM KILKA COUNTÓW DLA TEGO SAMEGO NUMERU LOTU NP. LOT1***
***

In [104]:
top_route_df = pd.DataFrame(flight_df_merged_2[['op_carrier_fl_num','origin_city_name']].\
                            value_counts()\
                            .reset_index())
top_route_df.rename(columns = {'op_carrier_fl_num':'count_flights'})
#                            .sort_values('count_flights',  ascending= False)\

,count_flights,origin_city_name,0
0,573,"Denver, CO",690
1,1027,"Boston, MA",642
2,1153,"Atlanta, GA",618
3,420,"Denver, CO",609
4,2127,"Boston, MA",598
...,...,...,...
25638,2745,"Atlanta, GA",1
25639,4298,"Denver, CO",1
25640,5171,"San Francisco, CA",1
25641,2747,"San Diego, CA",1


In [76]:
top_route_df = flight_df_merged_2.groupby(by =['op_carrier_fl_num','origin_airport_id'])['dest_city_name'].count()
# rename(columns ={'index':'flight','origin_airport_id':'flight_count'}).sort_values(by = 'flight_count',ascending = False)
# top_route_df = top_route_df.loc[top_route_df['flight_count'] >500]
top_route_df

op_carrier_fl_num  origin_airport_id
1                  11259                238
2                  11259                  3
                   11292                  5
                   12191                 53
                   12889                  1
                                       ... 
7664               14771                  1
7756               14107                  1
7851               11292                  1
7880               13303                  1
7881               12953                  2
Name: dest_city_name, Length: 25688, dtype: int64

In [37]:
top_airports_destination_df = flight_df_merged_2.groupby('dest_city_name').agg({'month':'count'}).reset_index()
top_airports_destination_df['dest_airport_ratio'] = top_airports_destination_df['month'] /\
                                                flight_04_03_df.shape[0]
# top_airports_destination_df
top_airports_destination_df.rename(columns = {'month':'flight'})
top_airports_destination_df.sort_values('dest_airport_ratio',  ascending = False).head(10)

,dest_city_name,month,dest_airport_ratio
1,"Atlanta, GA",122945,0.119740
6,"Chicago, IL",100333,0.097717
19,"Los Angeles, CA",75040,0.073084
10,"Denver, CO",64602,0.062918
24,"New York, NY",62972,0.061330
3,"Boston, MA",56057,0.054596
36,"Seattle, WA",50230,0.048921
33,"San Francisco, CA",49999,0.048696
9,"Dallas/Fort Worth, TX",39488,0.038459
17,"Las Vegas, NV",38494,0.037490


 ### Sprawdzenie dla `top_airport_origin`

### Sprawdzenie dla `top_airport_destination`

In [22]:
top_airports_destination_head = (top_airports_destination_df
                                 .sort_values(ascending=False)
                                 .head()
                                 .to_list()
                                 )
top_airports_destination_head = tuple(top_airports_destination_head)
top_airports_destination_head_expected = (122945, 100333, 87776, 64602, 57686)

assert top_airports_destination_head == top_airports_destination_head_expected, f"Nie zgadza się top 5 wierszy, oczekiwano wyników: {top_airports_destination_head_expected} otrzymano: {top_airports_destination_head}"


TypeError: DataFrame.sort_values() missing 1 required positional argument: 'by'

 # Wzbogacenie o dane pogodowe
 Używając procedury `read_sql_table`, wczytaj tabelę `airport_weather` do ramki `airport_weather_df`. Następnie wykonaj następujące polecenia:  
 1. Pozostaw w ramce tylko następujące kolumny: `['station', 'name', 'date', 'prcp', 'snow', 'snwd', 'tmax', 'awnd']`.  
 1. Połącz ramki `airport_list_df` wraz z `airport_weather_df` po odpowiedniej kolumnie używając takiego złączenia, aby w wyniku usunąć te wiersze (lotniska), które nie posiadają danych pogodowych. Dodatkowo, upewnij się, że zostanie tylko dodana kolumna `origin_airport_id`.

 Tutaj wczytaj ramkę `airport_weather`

In [ ]:
airport_list_df = pd.read_sql('airport_list',connection)
airport_list_df.info()

In [ ]:
airport_weather_df = pd.read_sql('airport_weather',connection)
airport_weather_df.info()

 Tutaj oczyść ramkę `airport_weather_df` z nadmiarowych kolumn

In [ ]:
# Pozostaw w ramce tylko następujące kolumny: ['station', 'name', 'date', 'prcp', 'snow', 'snwd', 'tmax', 'awnd']
columns_to_stay = ['station', 'name', 'date', 'prcp', 'snow', 'snwd', 'tmax', 'awnd']
columns_to_drop = [column for column in airport_weather_df.columns if column not in  columns_to_stay]
airport_weather_df.drop(columns_to_drop,  axis = 1, inplace = True)
airport_weather_df

 Tutaj połącz ramki `airport_list_df` oraz `airport_weather_df` aktualizując `airport_weather_df`

In [ ]:
airport_weather_df = pd.merge(airport_weather_df, airport_list_df['name'], on = 'name', how = 'inner',  suffixes = None)
airport_weather_df

In [ ]:
airport_weather_df.info()

 ### Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
airport_weather_df_expected_shape = (43394, 9)
airport_weather_df_shape = airport_weather_df.shape

assert airport_weather_df_expected_shape == airport_weather_df_shape, \
  f'Nieodpowiedni wymiar ramki airport_weather_df, oczekiwano (wierszy, kolumn): {airport_weather_df_expected_shape}'

# ***nie wiem dlaczego miałoby zostać 9 kolumn***

 ## Połączenie `airport_weather_df` oraz `flight_df`
 W celu złączenia ramek `airport_weather_df` oraz `flight_df` wykonaj następujące kroki:  
 1. w ramce `aiport_weather_df` występuje kolumna `date`, zrzutuj ją na typ `DATETIME`.  
 1. w ramce `flight_df` należy stworzyć nową kolumnę o nazwie `date`. W tym celu:  
 	- złącz kolumny `month`, `day_of_month` oraz `year` razem, użyj następującego formatu daty: `YYYY-MM-DD`.
 	- zrzutuj kolumnę `date` na typ `DATETIME`.  
 1. złącz ramki używając odpowiedniego klucza, wynik złączenia zapisz do ramki `flight_df`. Użyj złącznia typu `LEFT JOIN`.

 > Dlaczego istotne jest zachowanie typów przy złączeniu?

W trakcie pracy możesz posłużyć się następującymi artykułami z `LMS`:
 - `Python - analiza danych > Dzień 6 - Pandas > Merge`
 - `Python - analiza danych > Dzień 6 - Pandas > Praca z datetime`
 - Dokumentacje metody `to_datetime`: [klik](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)
 - Dostępne formaty dat: [klik](https://www.programiz.com/python-programming/datetime/strftime) - sekcja `Format Code List`

 Tutaj zrzutuj kolumnę `date` na `DATETIME` w ramce `airport_weather_df`

In [23]:
airport_weather_df['date'] = pd.to_datetime(airport_weather_df['date'])
airport_weather_df.info()

NameError: name 'airport_weather_df' is not defined

In [ ]:
# flight_df_merged_2['day_of_week'] = flight_df_merged_2['day_of_week'] .astype(str).str.zfill(2)
flight_df_merged_2 = flight_df_merged_2.rename(columns = {'day_of_week':'day'})
flight_df_merged_2

 Tutaj stwórz kolumnę `date` w ramce `flight_df`. Pamiętaj, aby była ona również typu `DATETIME`.

In [ ]:
flight_df_merged_2['date'] = pd.to_datetime(flight_df_merged_2[['year','month','day']])
flight_df_merged_2

 Tutaj złącz tabeli `airport_weather_df` oraz `flight_df`

In [ ]:
flight_df_merged_3 = pd.merge(flight_df_merged_2, airport_weather_df, how = 'left', on = 'date' )

In [ ]:
flight_df_merged_3 = flight_04_03_df
os.chdir('../data/processed')
flight_04_03_df = pd.to_csv('flight_df_04_03.csv')


In [ ]:
# testowe odczytanie zapisanego pliku
flight_04_03_df = pd.read_csv('flight_df_04_03.csv')
os.chdir('../../notebooks')

 ### Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
flight_df_expected_rows_amount = 1057391
assert flight_df.shape[0] == flight_df_expected_rows_amount, 'Ups, zmieniła się liczba wierszy...'


# Praca samodzielna
Używając `flight_df` zbadaj hipotezę o tym, że temperatura maksymalna wpływa na **odsetek** opóźnień lotów (kolumna `tmax`).  

Przy wykonywaniu tego zadania masz pełną dowolność, jednak powinno składać się conajmniej z następujących elementów:
- sprawdzenie, czy zmienna posiada obserwacje odstające,
- oczyszczenie danych o ile konieczne,
- przedstawienie w formie tabeli czy wzrost danej zmiennej powoduje zmianę w odsetku opóźnień lotów,
- wizualizację stworzonej wcześniej tabeli w formie wykresu,
- krótkiego opisu wyników w komórce markdown.

 ## Analiza dla kolumny `tmax`

In [ ]:
# nie podchodze do tej analizy

## Miejsce na Twój komentarz

# Podsumowanie
W tej części warsztatu dokonaliśmy kompleksowej analizy posiadanego zbioru danych. Eksploracja
pozwoliła nam na zapoznanie się z cechami charakterystycznymi lotów - wiemy już, które 
zmienne mogą mieć wpływ na opóźnienia lotów, a które nie. Co warto podkreślić, skupiliśmy się na wielu
aspektach tej analizy, co otwiera potencjalnie również inne możliwości dalszej pracy nad tą bazą.

W tym momencie przejdziemy do kolejnego kroku, w którym, na podstawie tej analizy, przygotujemy 
system raportowy. Zanim jednak stworzymy dashboard, potrzebujemy zaktualizować naszą bazę danych.